# Detailed Report

In [1]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm import load_node
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.calculation.work import WorkCalculation
from aiida.orm.calculation.job.quantumespresso.pw import PwCalculation
from aiida.orm.calculation.job.quantumespresso.pp import PpCalculation

import numpy as np
import bqplot as bq
import ipywidgets as ipw
from IPython.display import display, clear_output
import re
import gzip
import matplotlib.pyplot as plt
from collections import OrderedDict
import urlparse

In [2]:
def get_calc_by_label(workcalc, calc_class, label):
    qb = QueryBuilder()
    qb.append(WorkCalculation, filters={'uuid':workcalc.uuid})
    qb.append(calc_class, output_of=WorkCalculation, filters={'label':label})
    assert qb.count() == 1
    calc = qb.first()[0]
    assert(calc.get_state() == 'FINISHED')
    return calc

In [8]:
url = urlparse.urlsplit(jupyter_notebook_url)
#pk = urlparse.parse_qs(url.query)['pk'][0]
pk = 513
workcalc = load_node(pk=int(pk))

bands_calc = get_calc_by_label(workcalc, PwCalculation, "bands")
orbitals_calc = get_calc_by_label(workcalc, PpCalculation, "export_orbitals")
bands = bands_calc.out.output_band.get_bands()
structure = bands_calc.inp.structure
nspins, nkpoints, nbands = bands.shape

fermi_energy = workcalc.get_extra('fermi_energy')
vacuum_level = workcalc.get_extra('vacuum_level')
homo = workcalc.get_extra('homo')
lumo = workcalc.get_extra('lumo')
gap = workcalc.get_extra('gap')

In [115]:
def plot_spin(ispin):
    
    center = (homo + lumo)/2.0
    x_sc = bq.LinearScale()
    y_sc = bq.LinearScale(max=center+3.0, min=center-3.0)
    
    color_sc = bq.ColorScale(colors=['gray', 'red'], min=0.0, max=1.0)
    colors = np.zeros(nbands)
    
    ax_x = bq.Axis(label='X', scale=x_sc, grid_lines='solid', num_ticks=4)
    ax_y = bq.Axis(label='Y', scale=y_sc, orientation='vertical', grid_lines='solid')

    x_data = np.linspace(0, 0.5, nkpoints)
    y_datas = bands[ispin,:,:].transpose() - vacuum_level
    
    lines = bq.Lines(x=x_data, y=y_datas, color=colors, animate=True,
                     scales={'x': x_sc, 'y': y_sc, 'color': color_sc})

    ratio = 0.25
    layout = ipw.Layout(height="800px", width="200px")
    
    m_fig = dict(left=30, top=60, bottom=60, right=30)
    fig = bq.Figure(axes=[ax_x, ax_y], marks=[lines], title='Spin %i'%ispin, 
                    layout=layout, fig_margin=m_fig,
                    min_aspect_ratio=ratio, max_aspect_ratio=ratio)

    #fig.layout.margin = "0px"
    
    def print_event(self, target):
        global selected_spin, selected_band 
        selected_spin = ispin
        selected_band = target['data']['index']
        on_selection_change()
        
            
    #lines.on_element_click(print_event)
    lines.on_hover(print_event)

    #lines.interactions = {'click': 'tooltip'}
    #fig.fig_margin = {'top': 60, 'right': 20, 'bottom': 60, 'left': 20}
    save_btn = ipw.Button(description="Download png")
    save_btn.on_click(lambda b: fig.save_png())
    layout = ipw.Layout(align_items="center", padding="0px", margin="0px")
    box = ipw.VBox([fig, save_btn], layout=layout)
    return box, lines

In [109]:
def read_cube(fn):
    lines = gzip.open(fn).readlines()
    header = np.fromstring("".join(lines[2:6]), sep=' ').reshape(4,4)
    #print(header)
    natoms, nx, ny, nz = header[:,0].astype(int)
    z0 = header[0,3] # z origin z origin
    dz = header[3,3] # z step size
    cube = np.fromstring("".join(lines[natoms+6:]), sep=' ').reshape(nx, ny, nz)
    return(cube, z0, dz)

In [110]:
def on_selection_change():
    global selected_cube_files
    with info_out:
        clear_output()
        print("selected spin: %d"%selected_spin)
        print("selected band: %d"%selected_band)

        colors = np.zeros((nspins, nbands))
        colors[selected_spin, selected_band] = 1.0
        lines0.color = colors[0,:]
        lines1.color = colors[1,:]
        #orbitals_calc.out.retrieved.get_folder_list()
        
        lower = nkpoints * selected_spin
        upper = lower + nkpoints
        selected_cube_files = []
        for fn in sorted(orbitals_calc.out.retrieved.get_folder_list()):
            m = re.match("aiida.filplot_K(\d\d\d)_B(\d\d\d)_orbital.cube.gz", fn)
            if not m:
                continue
            k, b = int(m.group(1)), int(m.group(2))
            if b != selected_band:
                continue
            if lower < k and k < upper:
                selected_cube_files.append(fn)

        n = len(selected_cube_files)
        kpoint_slider.max = max(n, 1)
        print("found %d cube files"%n)
        on_kpoint_change(None)    

In [111]:
def on_kpoint_change(c):
    global selected_cube
    with kpnt_out:
        clear_output()
        i = kpoint_slider.value
        if i > len(selected_cube_files):
            print("Found no cube files")
            selected_cube = None
            height_slider.options = {"---":0}
            
        else:    
            fn = selected_cube_files[i-1]
            print(fn)
            absfn = orbitals_calc.out.retrieved.get_abs_path(fn)
            selected_cube, z0, dz = read_cube(absfn)
            nz = selected_cube.shape[2]
        
            zmid = structure.cell_lengths[2] / 2.0
            options = OrderedDict()
            for i in range(nz):
                z = (z0 + dz*i) * 0.529177 - zmid
                options[u"%.3f Å"%z] = i
            height_slider.options = options
        
        on_height_change(None)

In [112]:
def on_height_change(c):
    with orb_out:
        clear_output()
        if selected_cube is None:
            return
        plt.gcf().dpi=150.0
        z = height_slider.value
        plt.imshow(selected_cube[:,:,z].transpose())
        #plt.colorbar()
        plt.show()

In [114]:
box0, lines0 = plot_spin(0)
box1, lines1 = plot_spin(1)

info_out = ipw.Output()
kpnt_out = ipw.Output()
orb_out = ipw.Output()

layout = ipw.Layout(width="400px")
kpoint_slider = ipw.IntSlider(description="k-point", min=1, max=1, layout=layout)
kpoint_slider.observe(on_kpoint_change, names='value')

height_slider = ipw.SelectionSlider(description="height", options={"---":0}, layout=layout)
height_slider.observe(on_height_change, names='value')

layout = ipw.Layout(align_items="center")
side_box = ipw.VBox([info_out, kpoint_slider, height_slider, kpnt_out, orb_out], layout=layout)
display(ipw.HBox([box0, box1, side_box]))

{'top': 60, 'right': 30, 'bottom': 60, 'left': 30}
{'top': 60, 'right': 30, 'bottom': 60, 'left': 30}


A Jupyter Widget